# Loading and preparing images

This is a tutorial on how to load images and make them ready for downstream analysis.

The images has to be binary .tiff files and are therefore assumed to be thresholded already (black (zero) if no signal is detected and white (one) if signal is detected). The .tiff files can contain multiple layers, or there can be a .tiff for each layer. Both are accepted, more on this later.

### The example images can be downloaded from the following links:
Four species biofilm (for quantification and layer-based analysis):
https://github.com/Russel88/RCon3D/blob/master/ExampleData.zip

Two species in alginate beads (for finding aggregates):
https://github.com/Russel88/RCon3D/blob/master/ExampleData2.7z

The last is the one used in this example

### Load package

In [ ]:
library(RCon3D)

## Loading the images
Specify paths of images and set the working directory

In [ ]:
path <- "/Projects/Alginate beads/Images"
setwd(gsub("/Images","",path))

### Convert tiff to R-friendly RDS files
`loadIMG` creates the RDS files, and when we run the script again we can just use `findIMG` to get the paths for the RDS files.

`channels` should be unique and exclusive names in the filenames of the images. 

The following might give warnings about TIFF image tags that cannot be read. This should not be a problem.

In [ ]:
img <- loadIMG(path, channels = c("C0","C1"))

Check that the number of images are as expected:

In [ ]:
print(img)

If the images has been loaded previously, we can use `findIMG` to get the paths:

In [ ]:
# img <- findIMG(path)

We test the dimensions of the loaded images to ensure they are as expected:

In [ ]:
lapply(1:length(img), function(x) dim(readRDS(img[x])))

`img` contains the paths for the RDS files, and this is the input for subsequent analyses.

## Prepare images (optional)
If the channels do not correspond to the microbial strains we need to prepare the images before further analysis.

In this example channel C1 stains both strains. C0 correspond to only one of the strains (Xanthomonas).
Xanthomonas is therefore the pixels where both C0 and C1 is detected. The other strain, Paenibacillus, is only the C1 pixels where C0 is not observed

#### Paenibacillus is C1 - C0

In [ ]:
img.pan <- merge_channels(path, img, channels = c("C1","C0"), method = "subtract")

#### Xanthomonas is C0 AND C1

In [ ]:
img.xan <- merge_channels(path, img, channels = c("C1","C0"), method = "intersect")

#### If they have already been created:

In [ ]:
# img.pan <- img[grep("RCon3D.C1.C0.subtract",img)]
# img.xan <- img[grep("RCon3D.C1.C0.intersect",img)]

Check that the paths are correct:

In [ ]:
print(img.pan)
print(img.xan)

## The null channel (optional)

For some analyses we might want arrays corresponding to all the empty space in the images. This could for example be used to estimate the amount of empty space around a certain channel. 

In [ ]:
img_empty <- create_nulls(img, channels = c("C0","C1"))

`img_empty` then contains the paths for the RDS files corresponding to the empty space, or more precisely, the space note occupied by either "C0" or "C1".